In [ ]:
# For a project, generate all the SD images. 
# Add to the album for each scene

# {sd_workflow: '1', no_images: '5', project_id: '17'}
# get all the scenes for each project, all the album_ids for each scene and the list of prompts for each scene



In [1]:

from funcs_supabase import *
import random
import comfyui
import media

def get_prompts_list(project_id):

    prompts_list = []

    conditions = [
    {"column": "project_id", "operator": "eq", "value": project_id},
    {"column": "status", "operator": "eq", "value": "Active"}
    ]
    result = select_data_with_conditions("scenes", conditions)

    for scene in result:
        sd_prompts_list = scene.get("sd_prompts")

        for prompt in sd_prompts_list:
            # print(prompt)

            scene["prompt"] = prompt

            prompts_list.append(scene)

    # print(prompts_list)
    return prompts_list




def get_prompt_payload(input_dict):

    rand_no = random.randint(1000, 1000000)
    default_dict = {
        'prompt': "a man in",
        "batch_size": 1,
        'seed': rand_no
        }

    final_dict2 = default_dict.copy()
    final_dict2.update(input_dict)

    # REPLACE THIS WITH ACTUAL TEMPLATE LATER
    filepath = "workflows/comfyui_default_template.txt"

    with open(filepath, 'r') as file:
        template_content = file.read()

    rendered_template = comfyui.render_from_template(template_content, final_dict2)
    return rendered_template



def main():
    project_id = 19
    seed = 100000096777
    batch_size = 10


    prompts_list = get_prompts_list(project_id)
    for list_item in prompts_list:
        # print(list_item)
        prompt = list_item.get("prompt")
        prompt = prompt.strip()
        scene_text = list_item.get("scene_text")
        album_id = list_item.get("album_id")

        my_dict = {
        'prompt': prompt,
        "batch_size": batch_size,
        'seed': seed
        }

        payload = get_prompt_payload(my_dict)
        # print(payload)

        images = comfyui.get_images(payload)
        # print("\n images --> \n ")
        # print(images)
        # print("\n")
        image_binaries_list = comfyui.collect_image_binaries(images)

        for image_b in image_binaries_list:
            
            # upload to supabase storage
            # print(image_b)
            bucket="media"
            img_url = media.upload_to_supabase(bucket, image_b, is_binary=True)
            print(img_url)

            # add to media table
            media_data = {
            "input_phrase": prompt,
            "type": "image",
            "source_type": "sd",
            "source_data": payload,
            "source": "workflow",
            "url": img_url,
            "bucket": "media",
            "description" : scene_text,
            }

            media_table_response = insert_data("media", media_data)
            media_id = media_table_response['id']
            print(media_id)

            # add to album media link table
            media_link_data = {
            "album_id": album_id,
            "media_id": media_id
            }

            table = "album_media_link"
            response = insert_data(table, media_link_data)
            print(response)








            


main()

ModuleNotFoundError: No module named 'funcs_supabase'

In [ ]:
# import actions
# media = actions.return_processed_media_list(119)
# print(media)

In [ ]:
import funcs_database
import actions
project =19
article = """Once upon a time, Emily faced the colossal mountain of student debt, feeling overwhelmed and suffocated by the numbers. Each bill in the mailbox served as a constant reminder of her inability to pursue her true passion: wildlife photography. Then one day, inspired by a lecture from a famous photographer, she took out her old camera and began capturing the beauty of the local forests. She used her small social media following to display her work, and soon enough, she attracted the attention of a local art gallery. The gallery offered to host an exhibition of her photography, and it was a resounding success. 
Using the funds from the sold pieces, Emily managed to pay off a significant chunk of her debt, liberating her to take more daring trips for even better shots. More exhibitions followed, each more successful than the last, until she was finally free of debt. With her newfound financial freedom, Emily established a wildlife sanctuary, combining her passion for photography and conservation. Her story spread, and soon she was invited to give lectures, inspiring the next generation to conquer their obstacles just as she had. Emily looked at her life, realizing that she had transformed her mountain of debt into a mountain of dreams, climbed and conquered.
"""
split= actions.split_article_into_scenes(project, article)
print(split)